In [3]:
import pandas as pd
import mariadb
import joblib

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

Database connection

In [4]:
conn = mariadb.connect(
    user="admin",
    password="admin",
    host="127.0.0.1",
    port=3306,
    database="grade_prediction"
)

Load Data

In [5]:
query = "SELECT * FROM students_training;"
data = pd.read_sql(query, conn)
data = data.drop(columns=['id'])
print(data.head())

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  internet romantic  famrel  freetime  goout Dalc Walc health absences  G3  
0       no       no       4         3      4    1    1      3        6   6  
1      yes       no       5         3      3    1    1      3        4   6  
2      yes       no       4         3      2    2    3      3       10  10  
3      yes      yes       3         2      2    1    1      5        2  15  
4       no       no       4         3      2    1    2      5        4  10  

[5 rows x 31 columns]


/tmp/ipykernel_22601/316138484.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


Seperate data into x and y

In [6]:
categorical_features = [
    "school", "sex", "address", "famsize", "Pstatus",
    "Mjob", "Fjob", "reason", "guardian",
    "schoolsup", "famsup", "paid", "activities",
    "nursery", "higher", "internet", "romantic"
]

numerical_features = [
    "age", "Medu", "Fedu", "traveltime", "studytime", "failures",
    "famrel", "freetime", "goout", "Dalc", "Walc", "health", "absences"
]

all_features = categorical_features + numerical_features

X = data[all_features]
y = data["G3"]

Train model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
], remainder="passthrough")

pipeline = Pipeline([
    ("pre", preprocessor),
    ("reg", RandomForestRegressor()) 
])

pipeline.fit(X_train, y_train)

/home/pascal/Projects/grade_prediction/.venv/lib/python3.10/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('pre',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['school', 'sex', 'address',
                                                   'famsize', 'Pstatus', 'Mjob',
                                                   'Fjob', 'reason', 'guardian',
                                                   'schoolsup', 'famsup',
                                                   'paid', 'activities',
                                                   'nursery', 'higher',
                                                   'internet', 'romantic'])])),
                ('reg', RandomForestRegressor())])

Test model

In [8]:
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📉 MSE: {mse:.2f}")
print(f"📊 R²: {r2:.2f}")

print("\n🔍 Beispielvorhersagen:")
for true, pred in list(zip(y_test, y_pred))[:10]:
    print(f"Actual: {true:.1f}  |  Predicted: {pred:.1f}")

📉 MSE: 13.57
📊 R²: 0.12

🔍 Beispielvorhersagen:
Actual: 11.0  |  Predicted: 9.7
Actual: 8.0  |  Predicted: 10.3
Actual: 13.0  |  Predicted: 10.6
Actual: 11.0  |  Predicted: 12.9
Actual: 12.0  |  Predicted: 12.5
Actual: 11.0  |  Predicted: 10.9
Actual: 0.0  |  Predicted: 3.1
Actual: 11.0  |  Predicted: 12.3
Actual: 15.0  |  Predicted: 11.6
Actual: 13.0  |  Predicted: 14.0


Save the machine learning model in its own file

In [9]:
joblib.dump(pipeline, "model.pkl")
print("\n Modell gespeichert unter model.pkl")


 Modell gespeichert unter model.pkl
